In [1]:
%load_ext Cython
import cProfile
import pandas as pd
import numpy as np
import math
import re
from itertools import combinations_with_replacement
from itertools import count
from itertools import tee

df = pd.read_csv('taxi.csv', index_col = 'Unnamed: 0')

In [2]:
def average_difference(month,year,dayofweek,hr,df):
    community_areas  = np.zeros(77)
    df = df.dropna()
    df = df[(df['month'] == month) & (df['year'] == year) & (df['dayofweek'] == dayofweek) & (df['hour'] == hr+1)]
    missing_ca = [i for i in range(1,78) if i not in df['pickup_community_area'].values]
        
    for i in missing_ca:
        df = df.append(pd.Series([0,month, year, dayofweek,hr,i], index=df.columns), ignore_index=True)
        
    for i in df['pickup_community_area'].values:
        diff_sum = 0
        record = df[df['pickup_community_area'] == i]
        
        for j in df['pickup_community_area'].values:
            diff_sum += record['count(1)'].values[0] - df[df['pickup_community_area'] == j]['count(1)'].values[0] 
        
        community_areas[int(i-1)] = diff_sum/77
        
    return community_areas
        

In [3]:
def twentyfourhr(month,year,dayofweek,df):
    day = np.zeros((24,77))
    for hr in range(0,24): 
        day[hr] = average_difference(11,2014,dayofweek,hr,df) 
    return day

In [4]:
cProfile.run('twentyfourhr(11,2014,7,df)')

         144089724 function calls (142487904 primitive calls) in 139.517 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   288947    0.285    0.000    0.405    0.000 <frozen importlib._bootstrap>:416(parent)
  1595133    1.300    0.000    2.606    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
       24    2.320    0.097  139.517    5.813 <ipython-input-2-05f00ac16e0f>:1(average_difference)
       24    0.013    0.001    0.030    0.001 <ipython-input-2-05f00ac16e0f>:5(<listcomp>)
        1    0.001    0.001  139.517  139.517 <ipython-input-3-ca35a0587f93>:1(twentyfourhr)
        1    0.000    0.000  139.517  139.517 <string>:1(<module>)
   146468    0.166    0.000    0.262    0.000 __init__.py:204(iteritems)
       24    0.000    0.000    0.000    0.000 __init__.py:210(itervalues)
        3    0.000    0.000    0.000    0.000 _collections_abc.py:252(__subclasshook__)
        2    0.000    0.000    0.000    0.000

       24    0.000    0.000    0.000    0.000 nanops.py:256(_na_ok_dtype)
       24    0.000    0.000    0.000    0.000 nanops.py:260(_view_if_needed)
       24    0.000    0.000    0.000    0.000 nanops.py:266(_wrap_results)
       24    0.001    0.000    1.228    0.051 nanops.py:328(nansum)
       96    0.000    0.000    0.000    0.000 nanops.py:64(check)
       24    0.016    0.001    1.245    0.052 nanops.py:69(_f)
       24    0.044    0.002    0.152    0.006 nanops.py:712(_maybe_null_out)
      120    0.000    0.000    0.000    0.000 nanops.py:72(<genexpr>)
   288960    0.057    0.000    0.057    0.000 numeric.py:110(is_all_dates)
   577200    1.302    0.000    3.130    0.000 numeric.py:2460(seterr)
   577200    1.121    0.000    1.225    0.000 numeric.py:2560(geterr)
   288600    0.306    0.000    0.389    0.000 numeric.py:2853(__init__)
   288600    0.373    0.000    2.337    0.000 numeric.py:2857(__enter__)
   288600    0.315    0.000    1.482    0.000 numeric.py:2862(__exit__

In [5]:
def twentyfourhr_opt(month,year,dayofweek,df):
    day = np.zeros((24,77))
    
    for hr in range(0,1): 
        community_areas  = np.zeros(77)
        df = df.dropna()
        df = df[(df['month'] == month) & (df['year'] == year) & (df['dayofweek'] == dayofweek) & (df['hour'] == hr+1)]
        missing_ca = [i for i in range(1,78) if i not in df['pickup_community_area'].values]
        
        for i in missing_ca:
            df = df.append(pd.Series([0,month, year, dayofweek,hr,i], index=df.columns), ignore_index=True)
        
        for i in df['pickup_community_area'].values:
            diff_sum = 0
            record = df[df['pickup_community_area'] == i]
        
            for j in df['pickup_community_area'].values:
                diff_sum += record['count(1)'].values[0] - df[df['pickup_community_area'] == j]['count(1)'].values[0] 
        
            community_areas[int(i-1)] = diff_sum/77
                      
        day[hr] = community_areas           
    
    return day
            
            
            

In [6]:
cProfile.run('twentyfourhr_opt(11,2014,7,df)')

         5996060 function calls (5929453 primitive calls) in 5.848 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12035    0.012    0.000    0.017    0.000 <frozen importlib._bootstrap>:416(parent)
    66397    0.054    0.000    0.108    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.096    0.096    5.848    5.848 <ipython-input-5-8fb46aa1e9af>:1(twentyfourhr_opt)
        1    0.001    0.001    0.001    0.001 <ipython-input-5-8fb46aa1e9af>:8(<listcomp>)
        1    0.000    0.000    5.848    5.848 <string>:1(<module>)
     6085    0.007    0.000    0.011    0.000 __init__.py:204(iteritems)
        1    0.000    0.000    0.000    0.000 __init__.py:210(itervalues)
        7    0.000    0.000    0.002    0.000 _decorators.py:135(wrapper)
        2    0.000    0.000    0.015    0.008 _methods.py:31(_sum)
    30101    0.016    0.000    0.135    0.000 _methods.py:37(_any)
       19    0.000    0.00

       19    0.000    0.000    0.000    0.000 missing.py:596(clean_reindex_fill_method)
       19    0.000    0.000    0.000    0.000 missing.py:74(clean_fill_method)
        1    0.000    0.000    0.000    0.000 nanops.py:179(_get_fill_value)
        1    0.000    0.000    0.039    0.039 nanops.py:202(_get_values)
        1    0.000    0.000    0.000    0.000 nanops.py:256(_na_ok_dtype)
        1    0.000    0.000    0.000    0.000 nanops.py:260(_view_if_needed)
        1    0.000    0.000    0.000    0.000 nanops.py:266(_wrap_results)
        1    0.000    0.000    0.056    0.056 nanops.py:328(nansum)
        4    0.000    0.000    0.000    0.000 nanops.py:64(check)
        1    0.001    0.001    0.057    0.057 nanops.py:69(_f)
        1    0.002    0.002    0.006    0.006 nanops.py:712(_maybe_null_out)
        5    0.000    0.000    0.000    0.000 nanops.py:72(<genexpr>)
    12040    0.002    0.000    0.002    0.000 numeric.py:110(is_all_dates)
    24050    0.055    0.000    0.132  

In [7]:
reg = twentyfourhr(11,2014,7,df)
opt = twentyfourhr_opt(11,2014,7,df)

In [8]:
for i in range(0,24):
    print(reg[i] == opt[i])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False]
[False False False False False False False False False False

In [9]:
reg[1]

array([-1.90142857e+02, -2.22142857e+02,  3.84857143e+02, -1.37142857e+02,
        2.85714286e+00,  3.20785714e+03,  2.45985714e+03,  7.41385714e+03,
       -2.86142857e+02, -2.73142857e+02, -2.65142857e+02, -2.83142857e+02,
       -2.79142857e+02, -2.27142857e+02, -2.71142857e+02, -1.92142857e+02,
       -2.72142857e+02, -2.86142857e+02, -2.75142857e+02, -2.87142857e+02,
       -2.06142857e+02,  6.25857143e+02, -2.79142857e+02,  1.82285714e+03,
       -2.81142857e+02, -2.87142857e+02, -2.84142857e+02,  1.06485714e+03,
       -2.87142857e+02, -2.85142857e+02, -2.08142857e+02,  8.42857143e+02,
       -1.18142857e+02, -2.64142857e+02, -2.82142857e+02, -2.86142857e+02,
       -2.88142857e+02, -2.88142857e+02, -2.78142857e+02, -2.87142857e+02,
       -2.70142857e+02, -2.87142857e+02, -2.87142857e+02, -2.87142857e+02,
       -2.88142857e+02, -2.88142857e+02, -2.88142857e+02, -2.88142857e+02,
       -2.88142857e+02, -2.88142857e+02, -2.87142857e+02, -2.88142857e+02,
       -2.88142857e+02, -

In [11]:
opt[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])